In [1]:
!pip install scrapy

  Using cached w3lib-1.22.0-py2.py3-none-any.whl (20 kB)
  Using cached cssselect-1.1.0-py2.py3-none-any.whl (16 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Created wheel for protego: filename=Protego-0.1.16-py3-none-any.whl size=7764 sha256=2585efebecf44326219066a25e8346393be4e1a29073dc427b202c044b4b9c7a
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\91\64\36\bd0d11306cb22a78c7f53d603c7eb74ebb6c211703bc40b686
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=12547 sha256=11fa83d0ab378c0bb1801a8175d8d7964dfaa1ee28474cf281a9f86faa698575
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\3c\31\7f\d7d7b5f0b9bad841ed856138ff0c5ee2bf2e04dbeb413097c8
Successfully built protego PyDispatcher


In [2]:
import scrapy

# 本爬蟲原來是爬 udemycoupon.discountsglobal.com ，因該網站已停止更新，拿來回收再利用
class Uspider03Spider(scrapy.Spider):
    name = 'uspider03'
    allowed_domains = ['www.onlinecourses24x7.com']
    start_urls = ['https://www.onlinecourses24x7.com/']
    
    COUNT_MAX = 15
    count = 0
    
    def parse(self, response):
        items = response.xpath('//*[@class="entry-title"]/a/@href').extract()  # 該頁面所有的文章連結
        for item in items:
            url = response.urljoin(item)
            yield scrapy.Request(url, callback=self.parse_url)
            
        if (self.count < self.COUNT_MAX):  # 如果沒超過設定上限，就繼續爬下一頁
            next_page_url = response.xpath('//*[@class="previous"]/a/@href').extract_first()
            yield scrapy.Request(next_page_url)
            
    def parse_url(self, response):  # 取得每一個 udemy 課程的 url
        title = response.xpath('//*/h1/text()').extract_first()
        
        time = response.xpath('//time[@class="entry-date published"]/@datetime').extract_first()
        time = time[:-15]  # "2021-05-21T18:39:18+01:00" to "2021-05-21"
        
        udemy_url = response.xpath('//*[@class="su-button-center"]/a/@href').extract_first()
        # 移除廣告碼
        udemy_url = udemy_url.replace('https://click.linksynergy.com/deeplink?id=KLRB8G/RnDE&mid=39197&u1=vini&murl=', '')
        udemy_url = udemy_url.replace('%3A', ':')
        udemy_url = udemy_url.replace('%2F', '/')
        udemy_url = udemy_url.replace('%3F', '?')
        udemy_url = udemy_url.replace('%3D', '=')

        tag_list = response.xpath('//*[@class="tag-links"]/a/text()').extract()
        tags = ''.join(str(e) for e in tag_list)  # from list to string 

        self.count = self.count + 1
            
        yield {
            'Count':  self.count,
            'Source':  "3",
            'Title': title,
            'Time': time,
            'UdemyURL': udemy_url,
            'Tags': tags
        }


2021-08-27 11:25:47 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-08-27 11:25:47 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.4.7, Platform Windows-10-10.0.19041-SP0
2021-08-27 11:25:47 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-08-27 11:25:47 [scrapy.crawler] INFO: Overridden settings:
{'SPIDER_LOADER_WARN_ONLY': True}
2021-08-27 11:25:47 [scrapy.extensions.telnet] INFO: Telnet Password: 9f37d6bf4b4e45b5
2021-08-27 11:25:47 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-08-27 11:25:47 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.ht